In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import os
import seaborn as sns
from scipy import stats
from madule.simulation import *

In [6]:
np.random.seed(123)

### Generate True Parameters For each Participant on Hierarchical level

In [7]:
"""Parameter recovery assessment.
True parameters for each participant are taken from hierarchical level parameters"""

# Number of simulation
simNumber = 1

# Set mean and std of unkown parameters from uniform distribution between 0 and 1
alphaAct_mu = np.round(np.random.rand(2, 2), 2)
alphaAct_sd = np.round(np.random.rand(2, 2), 2)

alphaClr_mu = np.round(np.random.rand(2, 2), 2)
alphaClr_sd = np.round(np.random.rand(2, 2), 2)

weghtAct_mu = np.round(np.random.rand(2, 2), 2)
weghtAct_sd = np.round(np.random.rand(2, 2), 2)

beta_mu = 5*np.round(np.random.rand(2, 2), 2)
beta_sd = 5*np.round(np.random.rand(2, 2), 2)

# call trueParam function to generate and save true parameters for each participant
trueParamAllParts(alphaAct_mu=alphaAct_mu, alphaAct_sd=alphaAct_sd,
          alphaClr_mu=alphaClr_mu, alphaClr_sd=alphaClr_sd,
          weghtAct_mu=weghtAct_mu, weghtAct_sd=weghtAct_sd,
          beta_mu=beta_mu, beta_sd=beta_sd,
          simNumber=simNumber)

All true parameters for each participant have been done successfully!


## Simulation RL Data

In [8]:
simulateActClrAllParts(simNumber=1)

All simulations have been done successfully!


/home/amingk/Documents/T7PD/ActStimRL/madule/simulation.py:10: RuntimeWarning: overflow encountered in exp
  denom = np.sum([nom + np.exp(values[1]*beta)])
/home/amingk/Documents/T7PD/ActStimRL/madule/simulation.py:9: RuntimeWarning: overflow encountered in exp
  nom = np.exp(values[0]*beta)
/home/amingk/Documents/T7PD/ActStimRL/madule/simulation.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return nom/denom
/home/amingk/Documents/T7PD/ActStimRL/madule/simulation.py:10: RuntimeWarning: overflow encountered in exp
  denom = np.sum([nom + np.exp(values[1]*beta)])
